In [3]:
import sys
import time
import numpy as np

# Default (desde Jupyter)
#N = 5 * 10**6
N = 10**7
# Si viene por línea de comandos
try:
    N = int(sys.argv[1])
except:
    pass


### Serial version with numpy w/ arrays

In [7]:
import sys
import numpy as np

def calc_pi_numpy(a, b):
    # 2. Calculate the squared distance from the origin for all points
    # This calculation (a**2 + b**2) is vectorized, 
    # meaning it's applied to all N elements simultaneously.
    dist_sq = a**2 + b**2
    
    # 3. Count the "hits" inside the circle
    # (dist_sq < 1.0) creates a boolean array (e.g., [True, False, True...])
    # np.sum() efficiently counts the True values (since True=1, False=0).
    M = np.sum(dist_sq < 1.0)
    
    # 4. Return the standard Monte Carlo estimate for pi
    return 4 * M / N
    
print(N)
#N = int(sys.argv[1])

# 1. Generate all N random coordinates at once
# Creates two arrays, x and y, each with N random numbers from -1 to 1
x = np.random.uniform(-1, 1, N).astype(np.float32)
y = np.random.uniform(-1, 1, N).astype(np.float32)
print(x.dtype)

pi = calc_pi_numpy(x,y)

print("\n \t Computing pi with numpy: \n")
%timeit -r3 calc_pi_numpy(x,y)
print("\t For %d trials, pi = %f\n" % (N,pi))


10000000
float32

 	 Computing pi with numpy: 

19.2 ms ± 17 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)
	 For 10000000 trials, pi = 3.141469



### Cupy

In [5]:
import cupy as cp

def calc_pi_cupy(x, y):
    # Copiamos datos a GPU
    xg = cp.asarray(x)
    yg = cp.asarray(y)

    # Cálculo en GPU
    dist_sq = xg*xg + yg*yg
    M = cp.sum(dist_sq < 1.0)

    # Traemos solo el escalar a CPU
    return 4.0 * float(M.get()) / N


print("\n \t Computing pi with CuPy:\n")

# Warm-up (obligatorio)
_ = calc_pi_cupy(x, y)
cp.cuda.Stream.null.synchronize()

# Medición REAL del tiempo
cp.cuda.Stream.null.synchronize()
%timeit -r3 calc_pi_cupy(x, y)
cp.cuda.Stream.null.synchronize()

pi_cupy = calc_pi_cupy(x, y)
print("\t For %d trials, pi = %f\n" % (N, pi_cupy))




 	 Computing pi with CuPy:

11.1 ms ± 20.8 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)
	 For 10000000 trials, pi = 3.141097



### Numba

In [8]:
from numba import vectorize

# Numba ufunc on GPU
@vectorize(['float32(float32, float32)'], target='cuda')
def inside_circle_numba(x, y):
    if x*x + y*y < 1.0:
        return 1.0
    else:
        return 0.0


def calc_pi_numba(x, y, N):
    # GPU: compute 0/1 array
    hits = inside_circle_numba(x, y)

    # CPU: reduction (as required by the statement)
    M = np.sum(hits)

    return 4.0 * M / N

# First call (includes JIT compilation)
pi_numba = calc_pi_numba(x, y, N)

print("\n \t Computing pi with Numba GPU: \n")
%timeit -r3 calc_pi_numba(x, y, N)
print("\t For %d trials, pi = %f\n" % (N, pi_numba))



 	 Computing pi with Numba GPU: 

20.1 ms ± 582 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)
	 For 10000000 trials, pi = 3.141469



### Selida ejecución en Bohr

### 10^7
10000000
float32

 	 Computing pi with numpy: 

18.1 ms ± 2.66 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)
	 For 10000000 trials, pi = 3.141621


 	 Computing pi with CuPy:

9.13 ms ± 15.5 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)
	 For 10000000 trials, pi = 3.141621


 	 Computing pi with Numba GPU: 

17.1 ms ± 38.5 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)
	 For 10000000 trials, pi = 3.141621
     
### 10^8
100000000
float32

 	 Computing pi with numpy: 

188 ms ± 63.4 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)
	 For 100000000 trials, pi = 3.141642


 	 Computing pi with CuPy:

91.7 ms ± 3.29 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)
	 For 100000000 trials, pi = 3.141642


 	 Computing pi with Numba GPU: 

154 ms ± 476 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)
	 For 100000000 trials, pi = 3.141640

## Análisis de resultados

Los resultados muestran que la versión con **NumPy en CPU** presenta un tiempo de ejecución que crece aproximadamente de forma lineal con el número de puntos N, como es esperable al tratarse de un cálculo vectorizado pero ejecutado únicamente en CPU.

El uso de **CuPy** permite aprovechar directamente la GPU, obteniéndose una reducción significativa del tiempo de ejecución respecto a NumPy, especialmente para valores grandes de N. Esto se debe a que tanto el cálculo vectorizado como la reducción se realizan completamente en la GPU, minimizando la transferencia de datos entre CPU y GPU.

La versión con **Numba utilizando @vectorize(target='cuda')** también permite ejecutar el cálculo en la GPU, pero en este caso el rendimiento es algo inferior al de CuPy (la reducción se realiza fuera del kernel). Aun así, el uso de Numba mejora los tiempos respecto a la versión puramente en CPU.

En conjunto, los resultados muestran que el uso de GPU acelera de forma notable este tipo de cálculos
científicos, siendo CuPy la opción más eficiente en este caso, mientras que Numba ofrece una alternativa
flexible aunque con un mayor coste de ejecución.